# Movie Genre Trends on IMDB over the years

### Create dataset as per genre ratings and review numbers

In [3]:
import pandas as pd

In [61]:
movie_general_data = pd.read_table('../data/general_data.tsv', delimiter='\t', usecols=['tconst','titleType', 'primaryTitle','genres', 'startYear'], dtype={'tconst':str,'titleType': str, 'primaryTitle':str, 'genres':object, 'startYear': object})
movie_rating_data = pd.read_table('../data/rating_data.tsv', delimiter='\t', usecols=['tconst','averageRating','numVotes'], dtype={'tconst':str,'averageRating':float, 'numVotes': int})
movie_data = pd.merge(movie_general_data, movie_rating_data, on='tconst', how='inner')

movie_data.query(expr = "titleType == 'movie'", inplace= True)

movie_data.drop(columns = ['tconst', 'titleType'], inplace= True)
movie_data.reset_index(inplace = True, drop = True)



In [68]:
movie_data.dropna(inplace=True, subset=['genres', 'averageRating','numVotes'])

null_string = '\\' + 'N'
movie_data.query(expr = 'genres != @null_string', inplace= True)

if isinstance(movie_data['genres'][0], str) :
    movie_data['genres'] = movie_data['genres'].map(lambda genre: genre.split(','))

movie_data_single_genre = movie_data.explode('genres', ignore_index= True)
display(movie_data_single_genre.head(10))


,primaryTitle,startYear,genres,averageRating,numVotes
0,Miss Jerry,1894,Romance,5.3,204
1,The Corbett-Fitzsimmons Fight,1897,Documentary,5.3,466
2,The Corbett-Fitzsimmons Fight,1897,News,5.3,466
3,The Corbett-Fitzsimmons Fight,1897,Sport,5.3,466
4,The Story of the Kelly Gang,1906,Action,6.0,820
5,The Story of the Kelly Gang,1906,Adventure,6.0,820
6,The Story of the Kelly Gang,1906,Biography,6.0,820
7,The Prodigal Son,1907,Drama,4.4,20
8,Robbery Under Arms,1907,Drama,4.3,24
9,Hamlet,1908,Drama,2.8,26
